In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('market-basket.csv', on_bad_lines='skip', sep=';', header=0)
data.head()

<ipython-input-3-e4230ffcd7b4>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('market-basket.csv', on_bad_lines='skip', sep=';', header=0)


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [4]:
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark  

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  object 
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 27.9+ MB


In [7]:
from pyspark.sql.types import *
df_schema = StructType([StructField("BillNo", StringType(), True)\
                       ,StructField("Itemname", StringType(), True)\
                       ,StructField("Quantity", IntegerType(), True)\
                       ,StructField("Date", StringType(), True)\
                       ,StructField("Price", StringType(), True)\
                       ,StructField("Customer", FloatType(), True)\
                       ,StructField("Country", StringType(), True)])

In [8]:
from google.colab import files
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth
import pandas


spark_data = spark.createDataFrame(data, schema=df_schema)

In [23]:
basket_data = spark_data.dropDuplicates(['BillNo', 'Itemname']).sort('BillNo')
basket_data = basket_data.groupBy("BillNo").agg(F.collect_list("Itemname")).sort('BillNo')

In [24]:
fp_Growth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.05, minConfidence=0.05) 
model = fp_Growth.fit(basket_data)

In [25]:
model.freqItemsets.show()
items = model.freqItemsets

+--------------------+----+
|               items|freq|
+--------------------+----+
|[WHITE HANGING HE...|2202|
|[JUMBO BAG RED RE...|2064|
|[REGENCY CAKESTAN...|1904|
|     [PARTY BUNTING]|1656|
|[LUNCH BAG RED RE...|1541|
|               [NaN]|1455|
|[ASSORTED COLOUR ...|1431|
|[SET OF 3 CAKE TI...|1346|
|[PACK OF 72 RETRO...|1279|
|[LUNCH BAG  BLACK...|1260|
|[NATURAL SLATE HE...|1232|
|[JUMBO BAG PINK P...|1211|
|[HEART OF WICKER ...|1181|
|[JUMBO STORAGE BA...|1178|
|[JUMBO SHOPPER VI...|1166|
|[LUNCH BAG SPACEB...|1147|
|[PAPER CHAIN KIT ...|1143|
|[LUNCH BAG CARS B...|1132|
|[JAM MAKING SET P...|1116|
|    [SPOTTY BUNTING]|1114|
+--------------------+----+
only showing top 20 rows



In [26]:
result_pdf = items.select("*").toPandas()
result_pdf.head()

,items,freq
0,[WHITE HANGING HEART T-LIGHT HOLDER],2202
1,[JUMBO BAG RED RETROSPOT],2064
2,[REGENCY CAKESTAND 3 TIER],1904
3,[PARTY BUNTING],1656
4,[LUNCH BAG RED RETROSPOT],1541


In [28]:
result_pdf.to_excel('result_pdfItemsFreq.xlsx')

In [29]:
fp_Growth2 = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.1, minConfidence=0.1) 
model2 = fp_Growth2.fit(basket_data)

In [30]:
model2.freqItemsets.show()
items2 = model2.freqItemsets

+--------------------+----+
|               items|freq|
+--------------------+----+
|[WHITE HANGING HE...|2202|
+--------------------+----+



In [31]:
result_pdf2 = items2.select("*").toPandas()
result_pdf2.head()

,items,freq
0,[WHITE HANGING HEART T-LIGHT HOLDER],2202


In [32]:
result_pdf.to_excel('result_pdfItemsFreq2.xlsx')

# There are 22 items with minsupport 0.05 and minconfidence 0.05
# There are only 1 item with minsupport 0.1 and minconfidence 0.1